In [ ]:
!pip install --upgrade google-cloud-aiplatform

In [ ]:
!pip install rich

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import pandas as pd
from rich.console import Console
console = Console()

In [ ]:
so_df = pd.read_csv('so_database_app.csv')
so_df.head()

,input_text,output_text,category
0,"python's inspect.getfile returns ""<string>""<p>...",<p><code>&lt;string&gt;</code> means that the ...,python
1,Passing parameter to function while multithrea...,<p>Try this and note the difference:</p>\n<pre...,python
2,How do we test a specific method written in a ...,"<p>Duplicate of <a href=""https://stackoverflow...",python
3,how can i remove the black bg color of an imag...,<p>The alpha channel &quot;disappears&quot; be...,python
4,How to extract each sheet within an Excel file...,<p>You need to specify the <code>index</code> ...,python


In [ ]:
import pickle

In [ ]:
with open('embeddings.pkl', 'rb') as file:
  question_embeddings = pickle.load(file)

In [ ]:
so_df['embeddings'] = question_embeddings.tolist()

In [ ]:
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances_argmin as distances_argmin

In [ ]:
query = ['How to concat dataframes pandas']

In [ ]:
import vertexai
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel

vertexai.init(project="devhack-3f0c2", location="us-central1")

model_name = "text-embedding-004"
task = "SEMANTIC_SIMILARITY"
dimensionality: int = 768
model = TextEmbeddingModel.from_pretrained(model_name)
kwargs = dict(output_dimensionality=dimensionality) if dimensionality else {}

query_embedding = model.get_embeddings(query, **kwargs)[0].values

In [ ]:
cos_sim_array = cosine_similarity([query_embedding],list(so_df.embeddings.values))

In [ ]:
index_doc_cosine = np.argmax(cos_sim_array)
index_doc_distances = distances_argmin([query_embedding],
                                       list(so_df.embeddings.values))[0]

In [ ]:
console.print(so_df.input_text[index_doc_cosine])

Concatenate 2 dataframes and repeat values from small one with pandas<p>I have these two dataframes:</p>
<div class="s-table-container">
<table class="s-table">
<thead>
<tr>
<th>Field1</th>
<th>Field2</th>
</tr>
</thead>
<tbody>
<tr>
<td>0.5</td>
<td>0.7</td>
</tr>
<tr>
<td>2</td>
<td>1</td>
</tr>
<tr>
<td>3</td>
<td>0.1</td>
</tr>
<tr>
<td>4</td>
<td>0.4</td>
</tr>
</tbody>
</table>
</div>
<p>and</p>
<div class="s-table-container">
<table class="s-table">
<thead>
<tr>
<th>Date</th>
<th>Time</th>
</tr>
</thead>
<tbody>
<tr>
<td>2022-08-01</td>
<td>1</td>
</tr>
<tr>
<td>2022-08-01</td>
<td>2</td>
</tr>
</tbody>
</table>
</div>
<p>and a I need to obtain the following:</p>
<div class="s-table-container">
<table class="s-table">
<thead>
<tr>
<th>Field1</th>
<th>Field2</th>
<th>Date</th>
<th>Time</th>
</tr>
</thead>
<tbody>
<tr>
<td>0.5</td>
<td>0.7</td>
<td>2022-08-01</td>
<td>1</td>
</tr>
<tr>
<td>2</td>
<td>1</td>
<td>2022-08-01</td>
<td>2</td>
</tr>
<tr>
<td>3</td>
<td>0.1</td>
<td>2022-08-01</td>
<td>1</td>
</tr>
<tr>
<td>4</td>
<td>0.4</td>
<td>2022-08-01</td>
<td>2</td>
</tr>
</tbody>
</table>
</div>
<p>Thanks in advance</p>

In [ ]:
console.print(so_df.output_text[index_doc_cosine])

<p>You can elongate your second dataframe to match dimentions, and then concatenate it with first dataframe.</p>
<pre class="lang-py prettyprint-override"><code>import pandas as pd

df1 = pd.DataFrame({'Field1': [0.5, 2, 3, 4], 'Field2': [0.7, 1, 0.1, 0.4]})
print(df1)
#    Field1  Field2
# 0     0.5     0.7
# 1     2.0     1.0
# 2     3.0     0.1
# 3     4.0     0.4

df2 = pd.DataFrame({'Date': ['2022-08-01', '2022-08-01'], 'Time': [1, 2]})
print(df2)
#          Date  Time
# 0  2022-08-01     1
# 1  2022-08-01     2

n = int(df1.size / df2.size)
df3 = pd.concat( * n, axis=0).reset_index(drop=True)
print(df3)
#          Date  Time
# 0  2022-08-01     1
# 1  2022-08-01     2
# 2  2022-08-01     1
# 3  2022-08-01     2

df4 = pd.concat(, axis=1)
print(df4)
#    Field1  Field2        Date  Time
# 0     0.5     0.7  2022-08-01     1
# 1     2.0     1.0  2022-08-01     2
# 2     3.0     0.1  2022-08-01     1
# 3     4.0     0.4  2022-08-01     2
</code></pre>
<p>or shorter:</p>
<pre class="lang-py prettyprint-override"><code>df4 = pd.concat([
    df1,
    pd.concat(
         * int(df1.size / df2.size),
        axis=0
    ).reset_index(drop=True)
], axis=1)
</code></pre>

In [ ]:
from vertexai.generative_models import GenerativeModel, Part
model = GenerativeModel(model_name="gemini-1.5-flash-001")

In [ ]:
context = "Question: " + so_df.input_text[index_doc_cosine] +\
"\n Answer: " + so_df.output_text[index_doc_cosine]

In [ ]:
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, \
             answer with \
             [I couldn't find a good match in the \
             document database for your query]
             """

In [ ]:
response = model.generate_content([prompt])
print(response.text)

The context provides a clear and concise explanation of how to concatenate two dataframes in pandas, specifically focusing on repeating values from the smaller dataframe to match the dimensions of the larger one. 

Here's a breakdown of the method presented:

1. **Create the DataFrames:** The code snippet initializes two dataframes, `df1` and `df2`, representing the example data provided in the question.

2. **Calculate Repetition Factor:** The code determines how many times the smaller dataframe (`df2`) needs to be repeated to match the size of the larger dataframe (`df1`). This is done by dividing the size of `df1` (number of elements) by the size of `df2`.

3. **Repeat and Reset Index:** The `pd.concat()` function is used to concatenate multiple copies of `df2`, the number of copies being determined by the repetition factor. The `reset_index(drop=True)` method ensures that the resulting dataframe has a continuous index.

4. **Concatenate Along Columns:** Finally, `pd.concat()` is us

In [ ]:
#create a function for downloading file with python
